In [1]:
import pandas as pd
import os
from main import get_miner
from log_parsing import from_id_to_template
import ast
from datetime import datetime

In [2]:
df = pd.read_csv(f"output/features_labeled.csv")

In [3]:
df.head()

,abnormal_time,inject_pod,inject_time,inject_type,normal_time,result_list_inhealthy_lessoften,result_list_inhealthy_moreoften,root_cause
0,2022-08-22 03:54,frontend-579b9bff58-t2dbm,2022-08-22 03:53:54,cpu_contention,2022-08-22 03:51,"[{'events': '23_106', 'score': 0.5400894187779...","[{'events': '76_4', 'score': 1.0, 'deepth': 1,...",['CpuUsageRate(%)']
1,2022-08-22 03:55,frontend-579b9bff58-t2dbm,2022-08-22 03:53:54,cpu_contention,2022-08-22 03:51,"[{'events': '76_197', 'score': 1.0, 'deepth': ...","[{'events': '76_4', 'score': 1.0, 'deepth': 1,...",['CpuUsageRate(%)']
2,2022-08-22 04:03,frontend-579b9bff58-t2dbm,2022-08-22 04:02:07,return,2022-08-22 03:51,"[{'events': '18_191', 'score': 1.0, 'deepth': ...","[{'events': '120_198', 'score': 1.0, 'deepth':...","['Serving product page started', 'GetProduct s..."
3,2022-08-22 04:04,frontend-579b9bff58-t2dbm,2022-08-22 04:02:07,return,2022-08-22 03:51,"[{'events': '18_191', 'score': 1.0, 'deepth': ...","[{'events': '120_198', 'score': 1.0, 'deepth':...","['Serving product page started', 'GetProduct s..."
4,2022-08-22 04:11,frontend-579b9bff58-t2dbm,2022-08-22 04:10:20,cpu_consumed,2022-08-22 03:51,"[{'events': '76_197', 'score': 1.0, 'deepth': ...","[{'events': '104_106', 'score': 0.605582524271...",['CpuUsageRate(%)']


In [4]:
mask = df.abnormal_time == "2022-08-22 04:45"
print(df[mask]['result_list_inhealthy_moreoften'].values)

["[{'events': '165_48', 'score': 1.0, 'deepth': 5, 'pod': 'paymentservice-9cdb6588f-554sm', 'resource_alert': 'NetworkP90(ms)', 'events_actual': '165_199'}, {'events': '158_65', 'score': 1.0, 'deepth': 4, 'pod': 'checkoutservice-578fcf4766-9csqn', 'resource_alert': 'CpuUsageRate(%)', 'events_actual': '158_197'}, {'events': '4_3', 'score': 0.5631067961165048, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '4_42'}, {'events': '104_106', 'score': 0.5342612419700214, 'deepth': 2, 'pod': 'currencyservice-cf787dd48-vpjrd', 'events_actual': '104_23'}, {'events': '4_50', 'score': 0.5294117647058824, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '4_42'}, {'events': '38_84', 'score': 0.5274542429284526, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm'}, {'events': '38_78', 'score': 0.518361581920904, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm'}, {'events': '4_38', 'score': 0.5169082125603864, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actua

In [5]:
df_test = df[mask]

In [6]:
df_test['result_list_inhealthy_lessoften'] = df_test['result_list_inhealthy_lessoften'].apply(lambda x: ast.literal_eval(x))
df_test['result_list_inhealthy_moreoften'] = df_test['result_list_inhealthy_moreoften'].apply(lambda x: ast.literal_eval(x))
df_test['root_cause'] = df_test['root_cause'].apply(lambda x: ast.literal_eval(x))

C:\Users\paul-\AppData\Local\Temp\ipykernel_37984\2024620825.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['result_list_inhealthy_lessoften'] = df_test['result_list_inhealthy_lessoften'].apply(lambda x: ast.literal_eval(x))
C:\Users\paul-\AppData\Local\Temp\ipykernel_37984\2024620825.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['result_list_inhealthy_moreoften'] = df_test['result_list_inhealthy_moreoften'].apply(lambda x: ast.literal_eval(x))
C:\Users\paul-\AppData\Local\Temp\ip

In [7]:
df_test['inject_type'].unique()

array(['cpu_contention'], dtype=object)

In [8]:
df_test['root_cause'].apply(str).unique()

array(["['CpuUsageRate(%)']"], dtype=object)

In [9]:
def get_timediff(inject_time, abnormal_time):
    inject_dt = datetime.strptime(inject_time, '%Y-%m-%d %H:%M:%S')
    abnormal_dt = datetime.strptime(abnormal_time, '%Y-%m-%d %H:%M')
    
    diff = max(1, abs(inject_dt.minute - abnormal_dt.minute))
    return diff

df_test["inject_comp_tdiff"] = df_test.apply(lambda row: get_timediff(row['inject_time'], row['abnormal_time']), axis=1)

C:\Users\paul-\AppData\Local\Temp\ipykernel_37984\4218699836.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test["inject_comp_tdiff"] = df_test.apply(lambda row: get_timediff(row['inject_time'], row['abnormal_time']), axis=1)


In [10]:
log_template_miner = get_miner()

In [18]:
def get_nezha_rank(result_list, root_cause, inject_pod):
    rank = 1
    rc_found_flag = False
            
    if len(root_cause) == 1:
        for i in range(len(result_list)):
            if "resource_alert" in result_list[i].keys():
                if str(root_cause[0]) in str(result_list[i]["resource_alert"]) and inject_pod in str(result_list[i]["pod"]):
                    rc_found_flag = True
                    break
            else:
                if "events_actual" in result_list[i].keys() and str(root_cause[0]) in from_id_to_template(int(result_list[i]["events_actual"].split("_")[1]),log_template_miner) and inject_pod in str(result_list[i]["pod"]):
                    rc_found_flag = True
                    break
                if i > 0:
                    if result_list[i-1]["score"] == result_list[i]["score"] and result_list[i-1]["deepth"] == result_list[i]["deepth"]:
                        continue
            rank += 1
            
    elif len(root_cause) == 2:
        for i in range(len(result_list)):
            if root_cause[0] in from_id_to_template(int(result_list[i]["events"].split(
                    "_")[0]),log_template_miner) and root_cause[1] in from_id_to_template(int(result_list[i]["events"].split("_")[1]),log_template_miner) and inject_pod in str(result_list[i]["pod"]):
                rc_found_flag = True
                break
            else:
                if i > 0:
                    # logger.info("%s, %s",
                    #             result_list[i-1]["score"], result_list[i]["score"])
                    if result_list[i-1]["score"] == result_list[i]["score"] and result_list[i-1]["deepth"] == result_list[i]["deepth"]:
                        continue
            rank += 1
    
    if not rc_found_flag:
        return -1
    return rank

In [19]:
df_test["nezha_rank_050"] = df_test.apply(lambda row: get_nezha_rank(row['result_list_inhealthy_moreoften'], row['root_cause'], row['inject_pod']), axis=1)

C:\Users\paul-\AppData\Local\Temp\ipykernel_37984\2384368545.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test["nezha_rank_050"] = df_test.apply(lambda row: get_nezha_rank(row['result_list_inhealthy_moreoften'], row['root_cause'], row['inject_pod']), axis=1)


In [20]:
pd.set_option('display.max_colwidth', 1000)

In [21]:
df_test

,abnormal_time,inject_pod,inject_time,inject_type,normal_time,result_list_inhealthy_lessoften,result_list_inhealthy_moreoften,root_cause,inject_comp_tdiff,nezha_rank_050
12,2022-08-22 04:45,checkoutservice-578fcf4766-9csqn,2022-08-22 04:44:16,cpu_contention,2022-08-22 03:51,"[{'events': '165_199', 'score': 1.0, 'deepth': 4, 'pod': 'paymentservice-9cdb6588f-554sm', 'resource_alert': 'NetworkP90(ms)'}, {'events': '158_197', 'score': 1.0, 'deepth': 3, 'pod': 'checkoutservice-578fcf4766-9csqn', 'resource_alert': 'CpuUsageRate(%)'}, {'events': '23_100', 'score': 0.723404255319149, 'deepth': 3, 'pod': 'currencyservice-cf787dd48-vpjrd'}, {'events': '24_100', 'score': 0.7, 'deepth': 3, 'pod': 'currencyservice-cf787dd48-vpjrd', 'events_actual': '24_99'}, {'events': '4_40', 'score': 0.5538461538461539, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '4_50'}, {'events': '4_42', 'score': 0.5272727272727272, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '4_50'}, {'events': '16_41', 'score': 0.5272727272727272, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '16_15'}, {'events': '23_106', 'score': 0.5132492113564668, 'deepth': 3, 'pod': 'currencyservice-cf787dd48-vpjrd'}, {'events': '120_198', 'score': 0.500802...","[{'events': '165_48', 'score': 1.0, 'deepth': 5, 'pod': 'paymentservice-9cdb6588f-554sm', 'resource_alert': 'NetworkP90(ms)', 'events_actual': '165_199'}, {'events': '158_65', 'score': 1.0, 'deepth': 4, 'pod': 'checkoutservice-578fcf4766-9csqn', 'resource_alert': 'CpuUsageRate(%)', 'events_actual': '158_197'}, {'events': '4_3', 'score': 0.5631067961165048, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '4_42'}, {'events': '104_106', 'score': 0.5342612419700214, 'deepth': 2, 'pod': 'currencyservice-cf787dd48-vpjrd', 'events_actual': '104_23'}, {'events': '4_50', 'score': 0.5294117647058824, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '4_42'}, {'events': '38_84', 'score': 0.5274542429284526, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm'}, {'events': '38_78', 'score': 0.518361581920904, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm'}, {'events': '4_38', 'score': 0.5169082125603864, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_act...",[CpuUsageRate(%)],1,2
